In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import os

# Load the datasets
data_path = "../data/"

# Load only the first 1000 rows of each dataset

dos_data = pd.read_csv(os.path.join(data_path, 'DoS_dataset.csv'), nrows=50000)
# fuzzy_data = pd.read_csv(os.path.join(data_path, 'Fuzzy_dataset.csv'), nrows=50000)
# gear_data = pd.read_csv(os.path.join(data_path, 'gear_dataset.csv'), nrows=50000)
# rpm_data = pd.read_csv(os.path.join(data_path, 'RPM_dataset.csv'), nrows=50000)


# Concatenate datasets
data = pd.concat([dos_data], axis=0)

# Separate numeric and categorical columns
numeric_features = data.select_dtypes(include=['float64', 'int64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

# Define preprocessing for numeric features (impute missing values with mean, scale them)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
    ('scaler', StandardScaler())])  # Normalize the numeric features

# Define preprocessing for categorical features (impute missing values with mode, one-hot encode them)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with the most frequent value (mode)
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])  # One-hot encode the categorical features

# Combine both transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Apply transformations to the data
data_preprocessed = preprocessor.fit_transform(data)

# Convert the processed data into a PyTorch tensor
data_tensor = torch.tensor(data_preprocessed, dtype=torch.float32)

# Create DataLoader
batch_size = 64
dataset = TensorDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Preprocessed Data Shape: {data_preprocessed.shape}")

# Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        # Encoder: Compressing input data
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        # Decoder: Reconstructing the original data
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_size),
            nn.Sigmoid()  # Using sigmoid to bring values between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the Autoencoder
input_size = data_preprocessed.shape[1]  # Number of features (columns) after preprocessing
model = Autoencoder(input_size)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for reconstruction error
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the Autoencoder
num_epochs = 50
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch[0]  # DataLoader returns a tuple
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Anomaly Detection: Detect anomalies based on reconstruction error
def detect_anomalies(data_loader, model, threshold):
    model.eval()  # Set the model to evaluation mode
    anomalies = []
    for batch in data_loader:
        inputs = batch[0]
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # If the reconstruction error exceeds the threshold, flag as anomaly
        if loss.item() > threshold:
            anomalies.append(inputs)
    return anomalies

# Set a threshold based on training loss (you can adjust this threshold)
threshold = 0.02  # Example threshold, fine-tune based on your data
anomalies = detect_anomalies(dataloader, model, threshold)

# Display the number of anomalies detected
print(f"Number of anomalies detected: {len(anomalies)}")


Preprocessed Data Shape: (50000, 699)
Epoch 1/50, Loss: 0.007477820385247469
Epoch 2/50, Loss: 0.005299842916429043
Epoch 3/50, Loss: 0.005933338776230812
Epoch 4/50, Loss: 0.005250704474747181
Epoch 5/50, Loss: 0.004941974300891161
Epoch 6/50, Loss: 0.006142771802842617
Epoch 7/50, Loss: 0.007173973601311445
Epoch 8/50, Loss: 0.004509979858994484
Epoch 9/50, Loss: 0.0053029670380055904


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import os

# Load the datasets
data_path = "../data/"

# Load only the first 1000 rows of each dataset

dos_data = pd.read_csv(os.path.join(data_path, 'DoS_dataset.csv'), nrows=90000) 
# fuzzy_data = pd.read_csv(os.path.join(data_path, 'Fuzzy_dataset.csv'),nrows=0000)
# gear_data = pd.read_csv(os.path.join(data_path, 'gear_dataset.csv'), nrows=50000)
# rpm_data = pd.read_csv(os.path.join(data_path, 'RPM_dataset.csv'), nrows=50000)


# Concatenate datasets
data = pd.concat([dos_data], axis=0)

# Separate numeric and categorical columns
numeric_features = data.select_dtypes(include=['float64', 'int64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

# Define preprocessing for numeric features (impute missing values with mean, scale them)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
    ('scaler', StandardScaler())])  # Normalize the numeric features

# Define preprocessing for categorical features (impute missing values with mode, one-hot encode them)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with the most frequent value (mode)
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])  # One-hot encode the categorical features

# Combine both transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Apply transformations to the data
data_preprocessed = preprocessor.fit_transform(data)

# Convert the processed data into a PyTorch tensor
data_tensor = torch.tensor(data_preprocessed, dtype=torch.float32)

# Create DataLoader
batch_size = 64
dataset = TensorDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Preprocessed Data Shape: {data_preprocessed.shape}")

# Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        # Encoder: Compressing input data
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        # Decoder: Reconstructing the original data
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_size),
            nn.Sigmoid()  # Using sigmoid to bring values between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the Autoencoder
input_size = data_preprocessed.shape[1]  # Number of features (columns) after preprocessing
model = Autoencoder(input_size)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for reconstruction error
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the Autoencoder
num_epochs = 50
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch[0]  # DataLoader returns a tuple
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Anomaly Detection: Detect anomalies based on reconstruction error
def detect_anomalies(data_loader, model, threshold):
    model.eval()  # Set the model to evaluation mode
    anomalies = []
    for batch in data_loader:
        inputs = batch[0]
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # If the reconstruction error exceeds the threshold, flag as anomaly
        if loss.item() > threshold:
            anomalies.append(inputs)
    return anomalies

# Set a threshold based on training loss (you can adjust this threshold)
threshold = 0.02  # Example threshold, fine-tune based on your data
anomalies = detect_anomalies(dataloader, model, threshold)

# Display the number of anomalies detected
print(f"Number of anomalies detected: {len(anomalies)}")

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import os
import torch.optim as optim

# Load the datasets
data_path = "../data/"

# Load only the first 1000 rows of each dataset

dos_data = pd.read_csv(os.path.join(data_path, 'DoS_dataset.csv'), nrows=10000) 
# fuzzy_data = pd.read_csv(os.path.join(data_path, 'Fuzzy_dataset.csv'),nrows=0000)
# gear_data = pd.read_csv(os.path.join(data_path, 'gear_dataset.csv'), nrows=50000)
# rpm_data = pd.read_csv(os.path.join(data_path, 'RPM_dataset.csv'), nrows=50000)

# Concatenate datasets
data = pd.concat([dos_data,], axis=0)

# Separate numeric and categorical columns
numeric_features = data.select_dtypes(include=['float64', 'int64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

# Define preprocessing for numeric features (impute missing values with mean, scale them)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
    ('scaler', StandardScaler())])  # Normalize the numeric features

# Define preprocessing for categorical features (impute missing values with mode, one-hot encode them)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with the most frequent value (mode)
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])  # One-hot encode the categorical features

# Combine both transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Apply transformations to the data
data_preprocessed = preprocessor.fit_transform(data)

# Convert the processed data into a PyTorch tensor
data_tensor = torch.tensor(data_preprocessed, dtype=torch.float32)

# Create DataLoader
batch_size = 64
dataset = TensorDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Preprocessed Data Shape: {data_preprocessed.shape}")

# Define the Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        # Encoder: Compressing input data
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        # Decoder: Reconstructing the original data
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_size),
            nn.Sigmoid()  # Using sigmoid to bring values between 0 and 1
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the Autoencoder
input_size = data_preprocessed.shape[1]  # Number of features (columns) after preprocessing
model = Autoencoder(input_size)

# Loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss for reconstruction error
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the Autoencoder
num_epochs = 50
for epoch in range(num_epochs):
    for batch in dataloader:
        inputs = batch[0]  # DataLoader returns a tuple
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Function to calculate reconstruction errors
def calculate_reconstruction_errors(data_loader, model):
    model.eval()  # Set the model to evaluation mode
    reconstruction_errors = []
    for batch in data_loader:
        inputs = batch[0]
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        reconstruction_errors.append(loss.item())  # Collect the reconstruction error for each batch
    return reconstruction_errors

# Step 2: Calculate reconstruction errors for the training data
reconstruction_errors = calculate_reconstruction_errors(dataloader, model)

# Step 3: Plot the reconstruction error distribution to help set the threshold
plt.hist(reconstruction_errors, bins=50)
plt.xlabel('Reconstruction Error')
plt.ylabel('Number of Samples')
plt.title('Distribution of Reconstruction Errors')
plt.show()

# Step 4: Dynamically calculate the threshold based on the mean and standard deviation of reconstruction errors
mean_error = np.mean(reconstruction_errors)
std_error = np.std(reconstruction_errors)

# Set threshold as mean + 2 standard deviations
threshold = mean_error + 2 * std_error
print(f"Dynamically calculated threshold based on data: {threshold}")

# You can also choose to use the 95th percentile as an alternative:
# threshold = np.percentile(reconstruction_errors, 95)
# print(f"Threshold based on 95th percentile: {threshold}")

# Function to detect anomalies and get their corresponding row indices
def detect_anomalies_with_indices(data_loader, model, threshold):
    model.eval()  # Set the model to evaluation mode
    anomalies = []
    anomaly_indices = []  # List to store indices of the anomalies
    idx = 0  # Index counter for the data rows
    for batch in data_loader:
        inputs = batch[0]
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        # If the reconstruction error exceeds the threshold, flag as anomaly
        if loss.item() > threshold:
            anomalies.append(inputs)
            anomaly_indices.append(idx)  # Store the index of the anomaly
        idx += 1  # Increment index counter
    return anomalies, anomaly_indices

# Detect anomalies and get their indices
anomalies, anomaly_indices = detect_anomalies_with_indices(dataloader, model, threshold)

# Print the number of anomalies detected and their corresponding row indices
print(f"Number of anomalies detected: {len(anomalies)}")
print(f"Anomalies detected at rows (indices): {anomaly_indices}")
